In [3]:
from pathlib import Path
from typing import Any, Dict
import numpy as np
import numpy.typing as npt
import pandas as pd
from lmfit import Parameters
from echopop import inversion
from echopop.nwfsc_feat import (
    # apportion,
    biology, 
    FEAT,
    ingest_nasc, 
    get_proportions, 
    kriging,
    load_data, 
    # mesh,
    spatial,
    transect, 
    utils,
    variogram
)
# =

In [4]:
# ==================================================================================================
# ==================================================================================================
# DEFINE DATA ROOT DIRECTORY
# --------------------------
DATA_ROOT = Path("C:/Users/Brandyn/Documents/GitHub/EchoPro_data/echopop_2019")
# DATA_ROOT = Path("C:/Users/Brandyn Lucca/Documents/Data/echopop_2019/")

# ==================================================================================================
# ==================================================================================================
# DATA INGESTION
# ==================================================================================================
# Organize NASC file
# ------------------

# Merge exports
df_intervals, df_exports = ingest_nasc.merge_echoview_nasc(
    nasc_path = DATA_ROOT / "raw_nasc/",
    filename_transect_pattern = r"T(\d+)",
    default_transect_spacing = 10.0,
    default_latitude_threshold = 60.0,
)

# ==================================================================================================
# Read in transect-region-haul keys
# ---------------------------------
TRANSECT_REGION_FILEPATH_ALL_AGES: Path = Path(
    DATA_ROOT / "Stratification/US_CAN_2019_transect_region_haul_age1+ auto_final.xlsx"
)
TRANSECT_REGION_SHEETNAME_ALL_AGES: str = "Sheet1"
TRANSECT_REGION_FILEPATH_NO_AGE1: Path = Path(
    DATA_ROOT / "Stratification/US_CAN_2019_transect_region_haul_age2+ auto_20191205.xlsx"
)
TRANSECT_REGION_SHEETNAME_NO_AGE1: str = "Sheet1"
TRANSECT_REGION_FILE_RENAME: dict = {
    "tranect": "transect_num",
    "region id": "region_id",
    "trawl #": "haul_num",
}

# Read in the transect-region-haul key files for each group
transect_region_haul_key_all_ages: pd.DataFrame = ingest_nasc.read_transect_region_haul_key(
    filename=TRANSECT_REGION_FILEPATH_ALL_AGES,
    sheetname=TRANSECT_REGION_SHEETNAME_ALL_AGES,
    rename_dict=TRANSECT_REGION_FILE_RENAME,
)

transect_region_haul_key_no_age1: pd.DataFrame = ingest_nasc.read_transect_region_haul_key(
    TRANSECT_REGION_FILEPATH_NO_AGE1, TRANSECT_REGION_SHEETNAME_NO_AGE1, TRANSECT_REGION_FILE_RENAME
)

# ==================================================================================================
# Read in transect-region-haul keys
# ---------------------------------
REGION_NAME_EXPR_DICT: Dict[str, dict] = {
    "REGION_CLASS": {
        "Age-1 Hake": "^(?:h1a(?![a-z]|m))",
        "Age-1 Hake Mix": "^(?:h1am(?![a-z]|1a))",
        "Hake": "^(?:h(?![a-z]|1a)|hake(?![_]))",
        "Hake Mix": "^(?:hm(?![a-z]|1a)|hake_mix(?![_]))",
    },
    "HAUL_NUM": {
        "[0-9]+",
    },
    "COUNTRY": {
        "CAN": "^[cC]",
        "US": "^[uU]",
    },
}

# Process the region name codes to define the region classes
# e.g. H5C - Region 2 corresponds to "Hake, Haul #5, Canada"
df_exports_with_regions: pd.DataFrame = ingest_nasc.process_region_names(
    df=df_exports,
    region_name_expr_dict=REGION_NAME_EXPR_DICT,
    can_haul_offset=200,
)

# ==================================================================================================
# [OPTIONAL] Generate transect-region-haul key from compiled values
# ---------------------------------

# Generate transect-region-haul key from compiled values
df_transect_region_haul_key_no_age1: pd.DataFrame = ingest_nasc.generate_transect_region_haul_key(
    df=df_exports_with_regions, 
    filter_list=["Hake", "Hake Mix"]
)

df_transect_region_haul_key_all_ages = ingest_nasc.generate_transect_region_haul_key(
    df=df_exports_with_regions, 
    filter_list=["Age-1 Hake", "Age-1", "Hake", "Hake Mix"]
)

# ==================================================================================================
# Consolidate the Echvoiew NASC export files
# ------------------------------------------
df_nasc_no_age1: pd.DataFrame = ingest_nasc.consolidate_echvoiew_nasc(
    df_merged=df_exports_with_regions,
    interval_df=df_intervals,
    region_class_names=["Hake", "Hake Mix"],
    impute_region_ids=True,
    transect_region_haul_key_df=transect_region_haul_key_no_age1,
)

df_nasc_all_ages: pd.DataFrame = ingest_nasc.consolidate_echvoiew_nasc(
    df_merged=df_exports_with_regions,
    interval_df=df_intervals,
    region_class_names=["Age-1 Hake", "Age-1", "Hake", "Hake Mix"],
    impute_region_ids=True,
    transect_region_haul_key_df=transect_region_haul_key_all_ages,
)

# ==================================================================================================
# [OPTIONAL] Read in a pre-consolidated NASC data file
# ----------------------------------------------------
FEAT_TO_ECHOPOP_COLUMNS: Dict[str, str] = {
    "transect": "transect_num",
    "region id": "region_id",
    "vessel_log_start": "distance_s",
    "vessel_log_end": "distance_e",
    "spacing": "transect_spacing",
    "layer mean depth": "layer_mean_depth",
    "layer height": "layer_height",
    "bottom depth": "bottom_depth",
    "assigned haul": "haul_num",
}

#
df_nasc_all_ages: pd.DataFrame = ingest_nasc.read_nasc_file(
    filename=DATA_ROOT / "Exports/US_CAN_NASC_2019_table_all_ages.xlsx",
    sheetname="Sheet1",
    column_name_map=FEAT_TO_ECHOPOP_COLUMNS,
)

In [5]:
# ==================================================================================================
# Load in the biolodical data
# ---------------------------
BIODATA_SHEET_MAP: Dict[str, str] = {
    "catch": "biodata_catch", 
    "length": "biodata_length",
    "specimen": "biodata_specimen",
}
SUBSET_DICT: Dict[Any, Any] = {
    "ships": {
        160: {
            "survey": 201906
        },
        584: {
            "survey": 2019097,
            "haul_offset": 200
        }
    },
    "species_code": [22500]
}
FEAT_TO_ECHOPOP_BIODATA_COLUMNS = {
    "frequency": "length_count",
    "haul": "haul_num",
    "weight_in_haul": "weight",
}
BIODATA_LABEL_MAP: Dict[Any, Dict] = {
    "sex": {
        1: "male",
        2: "female",
        3: "unsexed"
    }
}

# 
dict_df_bio = load_data.load_biological_data(
    biodata_filepath=DATA_ROOT / "Biological/1995-2023_biodata_redo.xlsx", 
    biodata_sheet_map=BIODATA_SHEET_MAP, 
    column_name_map=FEAT_TO_ECHOPOP_BIODATA_COLUMNS, 
    subset_dict=SUBSET_DICT, 
    biodata_label_map=BIODATA_LABEL_MAP
)

# ==================================================================================================
# Load in strata files
# --------------------
STRATA_SHEET_MAP = {
    "inpfc": "INPFC",
    "ks": "Base KS",
}
FEAT_TO_ECHOPOP_STRATA_COLUMNS = {
    "fraction_hake": "nasc_proportion",
    "haul": "haul_num",
    "stratum": "stratum_num",
}

#
df_dict_strata = load_data.load_strata(
    strata_filepath=DATA_ROOT / "Stratification/US_CAN strata 2019_final.xlsx", 
    strata_sheet_map=STRATA_SHEET_MAP, 
    column_name_map=FEAT_TO_ECHOPOP_STRATA_COLUMNS
)

# ==================================================================================================
# Load in geographical strata files
# ---------------------------------
GEOSTRATA_SHEET_MAP = {
    "inpfc": "INPFC",
    "ks": "stratification1",
}
FEAT_TO_ECHOPOP_GEOSTRATA_COLUMNS = {
    "latitude (upper limit)": "northlimit_latitude",
    "stratum": "stratum_num",
}

# 
df_dict_geostrata = load_data.load_geostrata(
    geostrata_filepath=DATA_ROOT / "Stratification/Stratification_geographic_Lat_2019_final.xlsx", 
    geostrata_sheet_map=GEOSTRATA_SHEET_MAP, 
    column_name_map=FEAT_TO_ECHOPOP_GEOSTRATA_COLUMNS
)

# ==================================================================================================
# Stratify data based on haul numbers
# -----------------------------------

# Add INPFC
# ---- NASC
df_nasc_all_ages = load_data.join_strata_by_haul(data=df_nasc_all_ages, 
                                                 strata_df=df_dict_strata["inpfc"],
                                                 stratum_name="stratum_inpfc") 
# ---- Biodata
dict_df_bio = load_data.join_strata_by_haul(dict_df_bio,
                                            df_dict_strata["inpfc"],
                                            stratum_name="stratum_inpfc")

# Add KS
# ---- NASC
df_nasc_all_ages = load_data.join_strata_by_haul(df_nasc_all_ages, 
                                                 df_dict_strata["ks"],
                                                 stratum_name="stratum_ks") 
df_nasc_no_age1 = load_data.join_strata_by_haul(df_nasc_no_age1, 
                                                df_dict_strata["ks"],
                                                stratum_name="stratum_ks") 
# ---- Biodata
dict_df_bio = load_data.join_strata_by_haul(dict_df_bio,
                                            df_dict_strata["ks"],
                                            stratum_name="stratum_ks") 

# ==================================================================================================
# Load kriging mesh file
# ----------------------

FEAT_TO_ECHOPOP_MESH_COLUMNS = {
    "centroid_latitude": "latitude",
    "centroid_longitude": "longitude",
    "fraction_cell_in_polygon": "fraction",
}

# 
df_mesh = load_data.load_mesh_data(
    mesh_filepath=DATA_ROOT / "Kriging_files/Kriging_grid_files/krig_grid2_5nm_cut_centroids_2013.xlsx", 
    sheet_name="krigedgrid2_5nm_forChu", 
    column_name_map=FEAT_TO_ECHOPOP_MESH_COLUMNS
)

# ==================================================================================================
# [OPTIONAL] Stratify data based on latitude intervals
# ----------------------------------------------------
# INPFC (from geostrata)
df_nasc_all_ages = load_data.join_geostrata_by_latitude(df_nasc_all_ages, 
                                                        df_dict_geostrata["inpfc"],
                                                        stratum_name="geostratum_inpfc")
# KS (from geostrata)
df_nasc_all_ages = load_data.join_geostrata_by_latitude(df_nasc_all_ages, 
                                                        df_dict_geostrata["ks"],
                                                        stratum_name="geostratum_ks")

# MESH
# ---- DataFrame merged with geographically distributed stratum number (KS or INPFC)
# -------- INPFC (from geostrata)
df_mesh = load_data.join_geostrata_by_latitude(df_mesh, 
                                               df_dict_geostrata["inpfc"], 
                                               stratum_name="geostratum_inpfc")
# -------- KS (from geostrata)
df_mesh = load_data.join_geostrata_by_latitude(df_mesh, 
                                               df_dict_geostrata["ks"], 
                                               stratum_name="geostratum_ks")

# ==================================================================================================
# Load kriging and variogram parameters
# -------------------------------------

FEAT_TO_ECHOPOP_GEOSTATS_PARAMS_COLUMNS = {
    "hole": "hole_effect_range",
    "lscl": "correlation_range",
    "nugt": "nugget",
    "powr": "decay_power",
    "ratio": "aspect_ratio",
    "res": "lag_resolution",
    "srad": "search_radius",
}

# 
dict_kriging_params, dict_variogram_params = load_data.load_kriging_variogram_params(
    geostatistic_params_filepath=(
        DATA_ROOT / "Kriging_files/default_vario_krig_settings_2019_US_CAN.xlsx"
    ),
    sheet_name="Sheet1",
    column_name_map=FEAT_TO_ECHOPOP_GEOSTATS_PARAMS_COLUMNS
)

# ==================================================================================================
# ==================================================================================================
# DATA PROCESSING
# ==================================================================================================
# Generate binned distributions [age, length]
# -------------------------------------------
AGE_BINS: npt.NDArray[np.number] = np.linspace(start=1., stop=22., num=22)
LENGTH_BINS: npt.NDArray[np.number] = np.linspace(start=2., stop=80., num=40)

# 
# ---- Length
utils.binify(
    data=dict_df_bio, bins=LENGTH_BINS, bin_column="length", 
)

# Age
utils.binify(
    data=dict_df_bio, bins=AGE_BINS, bin_column="age",
)

# ==================================================================================================
# Fit length-weight regression to the binned data
# -----------------------------------------------

# Dictionary for length-weight regression coefficients
dict_length_weight_coefs = {}

# For all fish
dict_length_weight_coefs["all"] = dict_df_bio["specimen"].assign(sex="all").groupby(["sex"]).apply(
    biology.fit_length_weight_regression,
    include_groups=False
)

# Sex-specific
dict_length_weight_coefs["sex"] = dict_df_bio["specimen"].groupby(["sex"]).apply(
    biology.fit_length_weight_regression,
    include_groups=False
)

# ==================================================================================================
# Compute the mean weights per length bin
# ---------------------------------------

# Sex-specific (grouped coefficients)
df_binned_weights_sex = biology.length_binned_weights(
    data=dict_df_bio["specimen"],
    length_bins=LENGTH_BINS,
    regression_coefficients=dict_length_weight_coefs["sex"],
    impute_bins=True,
    minimum_count_threshold=5
)

# All fish (single coefficient set)
df_binned_weights_all = biology.length_binned_weights(
    data=dict_df_bio["specimen"].assign(sex="all"),
    length_bins=LENGTH_BINS,
    regression_coefficients=dict_length_weight_coefs["all"],
    impute_bins=True,
    minimum_count_threshold=5,
)

# Combine the pivot tables by adding the "all" column to the sex-specific table
binned_weight_table = pd.concat([df_binned_weights_sex, df_binned_weights_all], axis=1)

# ==================================================================================================
# Compute the count distributions per age- and length-bins
# --------------------------------------------------------

# Dictionary for number counts
dict_df_counts = {}

# Aged
dict_df_counts["aged"] = get_proportions.compute_binned_counts(
    data=dict_df_bio["specimen"].dropna(subset=["age", "length", "weight"]), 
    groupby_cols=["stratum_ks", "length_bin", "age_bin", "sex"], 
    count_col="length",
    agg_func="size"
)

# Unaged
dict_df_counts["unaged"] = get_proportions.compute_binned_counts(
    data=dict_df_bio["length"].copy().dropna(subset=["length"]), 
    groupby_cols=["stratum_ks", "length_bin", "sex"], 
    count_col="length_count",
    agg_func="sum"
)

# ==================================================================================================
# Compute the number proportions
# ------------------------------
dict_df_number_proportion: Dict[str, pd.DataFrame] = get_proportions.number_proportions(
    data=dict_df_counts, 
    group_columns=["stratum_ks"],
    column_aliases=["aged", "unaged"],
    exclude_filters={"aged": {"sex": "unsexed"}},
)

# ==================================================================================================
# Distribute (bin) weight over age, length, and sex
# -------------------------------------------------
# Pre-allocate a dictionary
dict_df_weight_distr: Dict[str, Any] = {}

# Aged
dict_df_weight_distr["aged"] = get_proportions.binned_weights(
    length_dataset=dict_df_bio["specimen"],
    include_filter = {"sex": ["female", "male"]},
    interpolate_regression=False,
    contrast_vars="sex",
    table_cols=["stratum_ks", "sex", "age_bin"]
)

# Unaged
dict_df_weight_distr["unaged"] = get_proportions.binned_weights(
    length_dataset=dict_df_bio["length"],
    length_weight_dataset=binned_weight_table,
    include_filter = {"sex": ["female", "male"]},
    interpolate_regression=True,
    contrast_vars="sex",
    table_cols=["stratum_ks", "sex"]
)

# ==================================================================================================
# Calculate the average weights pre stratum when combining different datasets
# ---------------------------------------------------------------------------
df_averaged_weight = get_proportions.stratum_averaged_weight(
    proportions_dict=dict_df_number_proportion, 
    binned_weight_table=binned_weight_table,
    stratum_col="stratum_ks",
)

# ==================================================================================================
# Compute the length-binned weight proportions for aged fish
# ----------------------------------------------------------

# Initialize Dictionary container
dict_df_weight_proportion: Dict[str, Any] = {}

# Aged
dict_df_weight_proportion["aged"] = get_proportions.weight_proportions(
    weight_data=dict_df_weight_distr, 
    catch_data=dict_df_bio["catch"], 
    group="aged",
    stratum_col="stratum_ks"
)

# ==================================================================================================
# Compute the standardized haul weights for unaged fish
# -----------------------------------------------------

standardized_sexed_unaged_weights_df = get_proportions.scale_weights_by_stratum(
    weights_df=dict_df_weight_distr["unaged"], 
    reference_weights_df=dict_df_bio["catch"].groupby(["stratum_ks"])["weight"].sum(),
    stratum_col="stratum_ks",
)

# ==================================================================================================
# Compute the standardized weight proportionsfor unaged fish
# ----------------------------------------------------------

dict_df_weight_proportion["unaged"] = get_proportions.scale_weight_proportions(
    weight_data=standardized_sexed_unaged_weights_df, 
    reference_weight_proportions=dict_df_weight_proportion["aged"], 
    catch_data=dict_df_bio["catch"], 
    number_proportions=dict_df_number_proportion,
    binned_weights=binned_weight_table["all"],
    group="unaged",
    group_columns = ["sex"],
    stratum_col = "stratum_ks"
)


C:\Users\Brandyn\Documents\GitHub\echopop\echopop\nwfsc_feat\load_data.py:409: UserWarning: Dropping existing stratification columns {'year', 'nasc_proportion'} from the dataframe.
  warnings.warn(
C:\Users\Brandyn\Documents\GitHub\echopop\echopop\nwfsc_feat\load_data.py:409: UserWarning: Dropping existing stratification columns {'year', 'nasc_proportion'} from the dataframe.
  warnings.warn(
C:\Users\Brandyn\Documents\GitHub\echopop\echopop\nwfsc_feat\load_data.py:409: UserWarning: Dropping existing stratification columns {'year', 'nasc_proportion'} from the dataframe.
  warnings.warn(
C:\Users\Brandyn\Documents\GitHub\echopop\echopop\nwfsc_feat\load_data.py:409: UserWarning: Dropping existing stratification columns {'year', 'nasc_proportion'} from the dataframe.
  warnings.warn(


In [6]:
# ==================================================================================================
# ==================================================================================================
# NASC TO POPULATION ESTIMATE CONVERSION
# ==================================================================================================
# Initialize the Inversion class
# ------------------------------
MODEL_PARAMETERS = {
    "ts_length_regression": {
        "slope": 20.,
        "intercept": -68.
    },
    "stratify_by": ["stratum_ks"],
    "expected_strata": df_dict_strata["ks"].stratum_num.unique(),
    "impute_missing_strata": True,
    "haul_replicates": True,
}

# Initiate object to perform inversion
invert_hake = inversion.InversionLengthTS(MODEL_PARAMETERS)

# ==================================================================================================
# Invert number density
# ---------------------

# If the above haul-averaged `sigma_bs` values were calculated, then the inversion can can 
# completed without calling in additional biodata
df_nasc_all_ages = invert_hake.invert(df_nasc=df_nasc_all_ages,
                                      df_length=[dict_df_bio["length"], dict_df_bio["specimen"]])
df_nasc_no_age1 = invert_hake.invert(df_nasc=df_nasc_no_age1,
                                     df_length=[dict_df_bio["length"], dict_df_bio["specimen"]])
# ---- The average `sigma_bs` for each stratum can be inspected at:
invert_hake.sigma_bs_strata

# ==================================================================================================
# Set transect interval distances
# -------------------------------

# Calculate along-transect interval distances which is required for getting the area-per-interval 
# and therefore going from number density to abundance
transect.set_interval_distance(df_nasc=df_nasc_all_ages, interval_threshold=0.05)
transect.set_interval_distance(df_nasc=df_nasc_no_age1, interval_threshold=0.05)

# ==================================================================================================
# Calculate transect interval areas
# ---------------------------------
df_nasc_all_ages["area_interval"] = (
    df_nasc_all_ages["transect_spacing"] * df_nasc_all_ages["distance_interval"]
)
df_nasc_no_age1["area_interval"] = (
    df_nasc_no_age1["transect_spacing"] * df_nasc_no_age1["distance_interval"]
)

# ==================================================================================================
# Calculate remaining population metrics across all animals 
# ---------------------------------------------------------
biology.set_population_metrics(df_nasc=df_nasc_all_ages, 
                               metrics=["abundance", "biomass", "biomass_density"],
                               stratify_by="stratum_ks",
                               df_average_weight=df_averaged_weight["all"])

biology.set_population_metrics(df_nasc=df_nasc_no_age1, 
                               metrics=["abundance", "biomass", "biomass_density"],
                               stratify_by="stratum_ks",
                               df_average_weight=df_averaged_weight["all"])

# ==================================================================================================
# Get proportions for each stratum specific to age-1
# --------------------------------------------------

# Age-1 NASC proportions
age1_nasc_proportions = get_proportions.get_nasc_proportions_slice(
    number_proportions=dict_df_number_proportion["aged"],
    stratify_by=["stratum_ks"],
    ts_length_regression_parameters={"slope": 20., 
                                     "intercept": -68.},
    include_filter = {"age_bin": [1]}
)

# Age-1 number proportions
age1_number_proportions = get_proportions.get_number_proportions_slice(
    number_proportions=dict_df_number_proportion["aged"],
    stratify_by=["stratum_ks"],
    include_filter = {"age_bin": [1]}
)

# Age-1 weight proportions
age1_weight_proportions = get_proportions.get_weight_proportions_slice(
    weight_proportions=dict_df_weight_proportion["aged"],
    stratify_by=["stratum_ks"],
    include_filter={"age_bin": [1]},
    number_proportions=dict_df_number_proportion,
    length_threshold_min=10.0,
    weight_proportion_threshold=1e-10
)

# ==================================================================================================
# ==================================================================================================
# GEOSTATISTICS
# ==================================================================================================
# Load reference line (isobath)
# -----------------------------

df_isobath = load_data.load_isobath_data(
    isobath_filepath=DATA_ROOT / "Kriging_files/Kriging_grid_files/transformation_isobath_coordinates.xlsx", 
    sheet_name="Smoothing_EasyKrig", 
)

# ==================================================================================================
# Transform the geospatial coordinates for the transect data
# ----------------------------------------------------------
df_nasc_no_age1, delta_longitude, delta_latitude = spatial.transform_coordinates(
    data = df_nasc_no_age1,
    reference = df_isobath,
    x_offset = -124.78338,
    y_offset = 45.,   
)

# ==================================================================================================
# Transform the geospatial coordinates for the mesh data
# ------------------------------------------------------
df_mesh, _, _ = spatial.transform_coordinates(
    data = df_mesh,
    reference = df_isobath,
    x_offset = -124.78338,
    y_offset = 45.,   
    delta_x=delta_longitude,
    delta_y=delta_latitude
)


In [7]:
# - 
# Map to the variogram function API
# - 
VARIOGRAM_MODEL_PARAMETER_MAP = {
    "Bessel-Exponential": ["bessel", "exponential"],
    "Bessel-Gaussian": ["bessel", "gaussian"],
    "Cosine-Exponential": ["cosine", "exponential"],
    "Cosine-Gaussian": ["cosine", "gaussian"],
    "Cubic": "cubic",
    "Exponential": "exponential",
    "Exponential-Linear": ["exponential", "linear"],
    "Gaussian": "gaussian",
    "Gaussian-Linear": ["gaussian", "linear"],
    "J-Bessel": "jbessel",
    "K-Bessel": "kbessel",
    "Linear": "linear",
    "Matérn": "matern",
    "Nugget": "nugget",
    "Pentaspherical": "pentaspherical",
    "Power law": "power",
    "Rational quadratic": "quadratic",
    "Cardinal sine": "sinc",
    "Spherical": "spherical",
}

# - 
# Map to the Variogram initialization
# - 
BASE_VARIOGRAM_PARAMETER_MAP = {
    "n_lags": dict(name="Number of lags", widget="entry", step=1, string_format="d"),
    "lag_resolution": dict(name="Lag resolution", widget="entry", step=1e-4, string_format="0.4f")
}

In [135]:
import panel as pn
import holoviews as hv
import ast

In [144]:
pn.extension()
pn.config.comms = 'default'
hv.extension("bokeh", enable_mathjax=True)

# INSTRUCTIONS TAB

In [145]:
INSTRUCTIONS_TAB = pn.pane.Markdown("""
# Variogram Analysis Interactive GUI

## Overview
This interactive tool allows you to perform comprehensive variogram analysis with the following steps:

## Workflow

### Tab 2: Object Initialization
- Set the **lag resolution** and **number of lags** for the variogram analysis
- These parameters control the spatial resolution and extent of the analysis

### Tab 3: Compute Empirical Variogram
- Select the variable to analyze
- Configure azimuth filtering options
- Generate an interactive plot showing:
    - X-axis: Lag distance
    - Y-axis: Semivariance (γ)
    - Point size: Varies with lag count
    - Hover tooltips: Lag #, lag counts, and semivariance values

### Tab 4: Fit Theoretical Model
- Choose from available variogram models
- Parameters dynamically update based on selected model
- Black line shows theoretical fit on empirical plot

### Tab 5: Optimization
- Specify optimization parameters as dictionary
- Set parameter bounds and variation flags
- Red line shows optimized fit
- Display best-fit parameters with appropriate rounding

## Usage Notes
- Make sure to load your data before proceeding
- Each tab builds upon the previous steps
- Results can be saved for further analysis
""")


# OVERALL

In [146]:
def create_variogram_gui():
    """
    Main
    """
    tabs = pn.Tabs(
        ("Instructions", INSTRUCTIONS_TAB),
        dynamic = True
    )

    return tabs

In [147]:
gui = create_variogram_gui()
gui

AssertionError: 

Tabs(dynamic=True)
    [0] Markdown(str)

In [155]:
# Don't call pn.extension() yet
pn.config.comms = 'ipywidgets'  # Force ipywidgets backend
pn.extension()

# Try the simplest possible Panel object
app = pn.panel("Hello World")
app

AssertionError: 

Markdown(str)

In [157]:
!pip install jupyterlab_pyviz

ERROR: Could not find a version that satisfies the requirement jupyterlab_pyviz (from versions: none)

[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for jupyterlab_pyviz
c:\Users\Brandyn\miniconda3\envs\echopop_3_12\Lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedWriter name=4>
  return process_handler(cmd, _system_body)
c:\Users\Brandyn\miniconda3\envs\echopop_3_12\Lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=5>
  return process_handler(cmd, _system_body)
c:\Users\Brandyn\miniconda3\envs\echopop_3_12\Lib\site-packages\IPython\utils\_process_win32.py:124: ResourceWarning: unclosed file <_io.BufferedReader name=6>
  return process_handler(cmd, _system_body)


In [153]:
import panel as pn
pn.extension()

# Try without the pane wrapper
component = pn.Markdown("abcdefg " * 10)
component

AttributeError: module 'panel' has no attribute 'Markdown'

In [126]:
import panel as pn

ABOUT = """\
# About

[Panel](https://panel.pyviz.org/) is a powerfull framework for building **awesome analytics apps** in [Python](https://www.python.org/).

The purpose of this app is to test that a **multi-page Dashboard Layout** similar to the [bootstrap dashboard template](https://getbootstrap.com/docs/4.3/examples/dashboard/) from [getboostrap.com](https://getbootstrap.com/) can be implemented in [Panel](https://panel.pyviz.org/).
"""
about_sizing_mode_stretch_width = pn.pane.Markdown(
    ABOUT
)
about = pn.pane.Markdown(ABOUT)
about_width_policy_max = pn.pane.Markdown(ABOUT, width_policy="max")
app = pn.Column(
    about,
)
app.servable()

AssertionError: 

Column
    [0] Markdown(str)

In [113]:
INSTRUCTIONS_TAB

Exception ignored in: <function Widget.__del__ at 0x000001F3A0FC93A0>
Traceback (most recent call last):
  File "c:\Users\Brandyn\miniconda3\envs\echopop_3_12\Lib\site-packages\ipywidgets\widgets\widget.py", line 516, in __del__
    self.close()
  File "c:\Users\Brandyn\miniconda3\envs\echopop_3_12\Lib\site-packages\jupyter_bokeh\widgets.py", line 91, in close
    if self._document is not None:
       ^^^^^^^^^^^^^^
  File "c:\Users\Brandyn\miniconda3\envs\echopop_3_12\Lib\site-packages\jupyter_bokeh\widgets.py", line 81, in _document
    return self._model.document
           ^^^^^^^^^^^
AttributeError: 'BokehModel' object has no attribute '_model'


AssertionError: 

Markdown(str)

In [112]:
create_variogram_gui()

AssertionError: 

Tabs(dynamic=True)
    [0] Markdown(str)

In [ ]:
# ==================================================================================================
# ==================================================================================================
# INTERACTIVE VARIOGRAM GUI USING PANEL/HOLOVIEWS
# ==================================================================================================

import panel as pn
import holoviews as hv
import param
from lmfit import Parameters
import ast
import traceback
from echopop.nwfsc_feat.variogram_models import get_variogram_arguments, VARIOGRAM_MODELS

# Enable Panel extensions
pn.extension('tabulator', template='bootstrap')
hv.extension('bokeh')

class InteractiveVariogramGUI(param.Parameterized):
    """
    Interactive Variogram GUI using Panel and Holoviews for Jupyter notebooks
    """
    
    def __init__(self, **params):
        super().__init__(**params)
        
        # Initialize variogram object placeholder
        self.vgm = None
        self.empirical_plot = None
        self.theoretical_plot = None
        self.fitted_plot = None
        self.current_data = None
        self.current_variable = None
        
        # Map from display names to model names
        self.variogram_model_map = {
            "Bessel-Exponential": ["bessel", "exponential"],
            "Bessel-Gaussian": ["bessel", "gaussian"], 
            "Cosine-Exponential": ["cosine", "exponential"],
            "Cosine-Gaussian": ["cosine", "gaussian"],
            "Cubic": "cubic",
            "Exponential": "exponential",
            "Exponential-Linear": ["exponential", "linear"],
            "Gaussian": "gaussian",
            "Gaussian-Linear": ["gaussian", "linear"],
            "J-Bessel": "jbessel",
            "K-Bessel": "kbessel", 
            "Linear": "linear",
            "Matérn": "matern",
            "Nugget": "nugget",
            "Pentaspherical": "pentaspherical",
            "Power law": "power",
            "Rational quadratic": "quadratic",
            "Cardinal sine": "sinc",
            "Spherical": "spherical",
        }
        
        self.create_widgets()
        self.create_layout()
    
    def create_widgets(self):
        """Create all widgets for the interface"""
        
        # Tab 1: Instructions (markdown pane)
        self.instructions_pane = pn.pane.Markdown("""
        # Interactive Variogram Analysis Tool
        
        This tool provides a comprehensive interface for variogram analysis with the following workflow:
        
        ## Workflow Steps:
        
        1. **Initialization**: Set lag resolution and number of lags for variogram computation
        2. **Empirical Variogram**: Compute and visualize the empirical variogram from spatial data
        3. **Theoretical Fitting**: Fit theoretical variogram models to empirical data
        4. **Optimization**: Optimize model parameters using weighted least squares
        
        ## Usage Instructions:
        
        - Navigate through tabs sequentially for best results
        - Each tab builds upon previous computations
        - Hover over plots to see detailed information
        - Parameter values are validated in real-time
        
        ## Model Types Available:
        
        **Single Models**: Exponential, Gaussian, Spherical, Linear, Cubic, Power, Matérn, etc.
        
        **Composite Models**: Bessel-Exponential, Cosine-Gaussian, Exponential-Linear, etc.
        
        **Notes**: 
        - Point sizes in plots indicate number of lag pairs
        - Black line shows theoretical model, red line shows optimized fit
        - Parameters are automatically constrained based on model requirements
        """, width=800, height=500)
        
        # Tab 2: Initialization widgets
        self.lag_resolution_input = pn.widgets.NumberInput(
            name='Lag Resolution', value=0.002, start=0.0001, end=1.0, step=0.0001,
            format='0.4f', width=200
        )
        
        self.n_lags_input = pn.widgets.IntInput(
            name='Number of Lags', value=30, start=5, end=100, step=1, width=200
        )
        
        self.initialize_button = pn.widgets.Button(
            name='Initialize Variogram', button_type='primary', width=200
        )
        self.initialize_button.on_click(self.initialize_variogram)
        
        self.init_status = pn.pane.HTML(
            "<p><i>Click 'Initialize Variogram' to begin</i></p>", width=400
        )
        
        # Tab 3: Empirical variogram widgets
        self.variable_input = pn.widgets.TextInput(
            name='Variable Name', value='biomass_density', placeholder='e.g., biomass_density', 
            width=200
        )
        
        self.azimuth_filter = pn.widgets.Checkbox(name='Azimuth Filter', value=True)
        
        self.azimuth_threshold = pn.widgets.NumberInput(
            name='Azimuth Threshold', value=180.0, start=0, end=360, step=1.0, width=200
        )
        
        self.compute_empirical_button = pn.widgets.Button(
            name='Compute Empirical Variogram', button_type='primary', width=250
        )
        self.compute_empirical_button.on_click(self.compute_empirical_variogram)
        
        self.empirical_status = pn.pane.HTML(
            "<p><i>Initialize variogram first, then click 'Compute Empirical Variogram'</i></p>", 
            width=500
        )
        
        # Tab 4: Theoretical model widgets
        self.model_selector = pn.widgets.Select(
            name='Variogram Model', 
            options=list(self.variogram_model_map.keys()),
            value='Exponential',
            width=200
        )
        self.model_selector.param.watch(self.update_model_parameters, 'value')
        
        self.fit_theoretical_button = pn.widgets.Button(
            name='Fit Theoretical Model', button_type='primary', width=200
        )
        self.fit_theoretical_button.on_click(self.fit_theoretical_model)
        
        self.theoretical_params_pane = pn.Column(width=400)
        self.theoretical_status = pn.pane.HTML(
            "<p><i>Compute empirical variogram first</i></p>", width=500
        )
        
        # Tab 5: Optimization widgets
        self.optimization_input = pn.widgets.TextAreaInput(
            name='Optimization Parameters (dict)',
            value='{}',
            height=100, width=400,
            placeholder='e.g., {"max_nfev": 1000, "ftol": 1e-8}'
        )
        
        self.optimize_button = pn.widgets.Button(
            name='Optimize Parameters', button_type='primary', width=200
        )
        self.optimize_button.on_click(self.optimize_variogram)
        
        self.optimization_params_pane = pn.Column(width=600)
        self.optimization_status = pn.pane.HTML(
            "<p><i>Fit theoretical model first</i></p>", width=600
        )
        
        self.optimization_results = pn.pane.HTML("<p></p>", width=600)
        
        # Plot panes
        self.plot_pane = pn.pane.HoloViews(hv.Curve([]).opts(
            title="Variogram Plot", xlabel="Lag Distance", ylabel="Semivariance (γ)",
            width=600, height=400, tools=['hover', 'pan', 'wheel_zoom', 'reset']
        ))
    
    def create_layout(self):
        """Create the tabbed layout"""
        
        # Tab contents
        tab1 = pn.Column(self.instructions_pane, margin=10)
        
        tab2 = pn.Column(
            pn.pane.Markdown("## Variogram Initialization"),
            pn.Row(self.lag_resolution_input, self.n_lags_input),
            self.initialize_button,
            self.init_status,
            margin=10
        )
        
        tab3 = pn.Column(
            pn.pane.Markdown("## Empirical Variogram Computation"),
            pn.pane.Markdown("*Note: You must set `current_data` and call this manually with your DataFrame*"),
            pn.Row(self.variable_input, self.azimuth_filter, self.azimuth_threshold),
            self.compute_empirical_button,
            self.empirical_status,
            self.plot_pane,
            margin=10
        )
        
        tab4 = pn.Column(
            pn.pane.Markdown("## Theoretical Model Fitting"),
            self.model_selector,
            self.theoretical_params_pane,
            self.fit_theoretical_button,
            self.theoretical_status,
            margin=10
        )
        
        tab5 = pn.Column(
            pn.pane.Markdown("## Parameter Optimization"),
            self.optimization_input,
            self.optimization_params_pane,
            self.optimize_button,
            self.optimization_status,
            self.optimization_results,
            margin=10
        )
        
        # Create tabs
        self.tabs = pn.Tabs(
            ("Instructions", tab1),
            ("Initialization", tab2),
            ("Empirical Variogram", tab3),
            ("Theoretical Fitting", tab4),
            ("Optimization", tab5),
            dynamic=True
        )
        
        # Initialize model parameters for default selection
        self.update_model_parameters()
    
    def initialize_variogram(self, event=None):
        """Initialize the variogram object"""
        try:
            from echopop.nwfsc_feat.variogram import Variogram
            
            self.vgm = Variogram(
                lag_resolution=self.lag_resolution_input.value,
                n_lags=self.n_lags_input.value,
                coordinate_names=("x", "y")
            )
            
            self.init_status.object = f"""
            <p style="color: green;">
            <b>✓ Variogram initialized successfully!</b><br>
            Lag Resolution: {self.lag_resolution_input.value}<br>
            Number of Lags: {self.n_lags_input.value}
            </p>
            """
            
        except Exception as e:
            self.init_status.object = f"""
            <p style="color: red;">
            <b>✗ Initialization failed:</b><br>
            {str(e)}
            </p>
            """
    
    def set_data(self, data, variable=None):
        """Set the data for empirical variogram computation"""
        self.current_data = data
        if variable:
            self.variable_input.value = variable
            self.current_variable = variable
    
    def compute_empirical_variogram(self, event=None):
        """Compute empirical variogram"""
        if self.vgm is None:
            self.empirical_status.object = """
            <p style="color: red;"><b>✗ Please initialize variogram first</b></p>
            """
            return
            
        if self.current_data is None:
            self.empirical_status.object = """
            <p style="color: red;"><b>✗ No data set. Use set_data() method first</b></p>
            """
            return
        
        try:
            # Compute empirical variogram
            self.vgm.calculate_empirical_variogram(
                data=self.current_data,
                variable=self.variable_input.value,
                azimuth_filter=self.azimuth_filter.value,
                azimuth_angle_threshold=self.azimuth_threshold.value
            )
            
            # Create plot
            self.update_empirical_plot()
            
            self.empirical_status.object = f"""
            <p style="color: green;">
            <b>✓ Empirical variogram computed!</b><br>
            Variable: {self.variable_input.value}<br>
            Azimuth filter: {self.azimuth_filter.value}<br>
            Number of lags: {len(self.vgm.lags)}
            </p>
            """
            
        except Exception as e:
            self.empirical_status.object = f"""
            <p style="color: red;">
            <b>✗ Computation failed:</b><br>
            {str(e)}<br>
            {traceback.format_exc()}
            </p>
            """
    
    def update_empirical_plot(self):
        """Update the empirical variogram plot"""
        if self.vgm is None or self.vgm.gamma is None:
            return
        
        # Create scatter plot with hover information
        scatter_data = hv.Table({
            'lag_distance': self.vgm.lags,
            'semivariance': self.vgm.gamma,
            'lag_count': self.vgm.lag_counts,
            'lag_number': range(1, len(self.vgm.lags) + 1)
        })
        
        self.empirical_plot = scatter_data.to.scatter(
            'lag_distance', 'semivariance', 
            size=hv.dim('lag_count').norm() * 200 + 50,
            color='blue', alpha=0.7
        ).opts(
            title="Empirical Variogram",
            xlabel="Lag Distance", 
            ylabel="Semivariance (γ)",
            width=600, height=400,
            tools=['hover'],
            hover_tooltips=[
                ('Lag #', '@lag_number'),
                ('Lag Distance', '@lag_distance{0.4f}'),
                ('Semivariance', '@semivariance{0.4f}'),
                ('Lag Count', '@lag_count')
            ]
        )
        
        self.plot_pane.object = self.empirical_plot
    
    def update_model_parameters(self, event=None):
        """Update parameter inputs based on selected model"""
        model_name = self.model_selector.value
        model_spec = self.variogram_model_map[model_name]
        
        try:
            # Get model arguments
            params, _ = get_variogram_arguments(model_spec)
            
            param_widgets = []
            param_widgets.append(pn.pane.Markdown(f"### Parameters for {model_name}"))
            
            # Create parameter inputs based on model requirements
            self.theoretical_param_widgets = {}
            
            for param_name in params:
                if param_name == 'distance_lags':
                    continue
                    
                # Set default values based on parameter type
                default_value = self._get_default_param_value(param_name)
                
                widget = pn.widgets.NumberInput(
                    name=param_name.replace('_', ' ').title(),
                    value=default_value,
                    start=0.0 if param_name != 'decay_power' else 1.0,
                    step=0.01,
                    format='0.4f',
                    width=200
                )
                
                self.theoretical_param_widgets[param_name] = widget
                param_widgets.append(widget)
            
            self.theoretical_params_pane.clear()
            self.theoretical_params_pane.extend(param_widgets)
            
        except Exception as e:
            self.theoretical_params_pane.clear()
            self.theoretical_params_pane.append(
                pn.pane.HTML(f"<p style='color: red;'>Error loading parameters: {e}</p>")
            )
    
    def _get_default_param_value(self, param_name):
        """Get default values for variogram parameters"""
        defaults = {
            'nugget': 0.1,
            'sill': 1.0,
            'correlation_range': 10.0,
            'hole_effect_range': 5.0,
            'decay_power': 1.5,
            'aspect_ratio': 1.0,
            'shape_parameter': 1.0,
            'anisotropy_angle': 0.0
        }
        return defaults.get(param_name, 1.0)
    
    def fit_theoretical_model(self, event=None):
        """Fit theoretical model to empirical data"""
        if self.vgm is None or self.vgm.gamma is None:
            self.theoretical_status.object = """
            <p style="color: red;"><b>✗ Please compute empirical variogram first</b></p>
            """
            return
        
        try:
            # Create lmfit Parameters from widget values
            model_params = Parameters()
            
            for param_name, widget in self.theoretical_param_widgets.items():
                model_params.add(param_name, value=widget.value, min=0.0)
            
            # Get model specification
            model_spec = self.variogram_model_map[self.model_selector.value]
            
            # Fit theoretical model (without optimization yet)
            from echopop.nwfsc_feat.variogram_models import variogram
            
            # Calculate theoretical values
            theoretical_gamma = variogram(
                distance_lags=self.vgm.lags,
                model=model_spec,
                **{name: widget.value for name, widget in self.theoretical_param_widgets.items()}
            )
            
            # Update plot with theoretical line
            self.update_plot_with_theoretical(theoretical_gamma)
            
            self.theoretical_status.object = f"""
            <p style="color: green;">
            <b>✓ Theoretical model fitted!</b><br>
            Model: {self.model_selector.value}<br>
            Parameters: {len(self.theoretical_param_widgets)} parameters set
            </p>
            """
            
        except Exception as e:
            self.theoretical_status.object = f"""
            <p style="color: red;">
            <b>✗ Theoretical fitting failed:</b><br>
            {str(e)}<br>
            {traceback.format_exc()}
            </p>
            """
    
    def update_plot_with_theoretical(self, theoretical_gamma, fitted_gamma=None):
        """Update plot with theoretical and optionally fitted lines"""
        if self.empirical_plot is None:
            return
        
        # Create theoretical line
        theoretical_curve = hv.Curve(
            (self.vgm.lags, theoretical_gamma), 
            label='Theoretical'
        ).opts(color='black', line_width=2)
        
        combined_plot = self.empirical_plot * theoretical_curve
        
        # Add fitted line if available
        if fitted_gamma is not None:
            fitted_curve = hv.Curve(
                (self.vgm.lags, fitted_gamma), 
                label='Optimized'
            ).opts(color='red', line_width=2, line_dash='dashed')
            
            combined_plot = combined_plot * fitted_curve
        
        combined_plot = combined_plot.opts(
            hv.opts.Curve(show_legend=True),
            hv.opts.Overlay(legend_position='top_right')
        )
        
        self.plot_pane.object = combined_plot
    
    def optimize_variogram(self, event=None):
        """Optimize variogram parameters"""
        if self.vgm is None or self.vgm.gamma is None:
            self.optimization_status.object = """
            <p style="color: red;"><b>✗ Please fit theoretical model first</b></p>
            """
            return
        
        try:
            # Parse optimization parameters
            opt_dict_str = self.optimization_input.value.strip()
            if opt_dict_str == '':
                opt_dict_str = '{}'
            opt_kwargs = ast.literal_eval(opt_dict_str)
            
            # Create parameter widgets for optimization control
            self.create_optimization_controls()
            
            # Create lmfit Parameters with vary/min/max controls
            model_params = Parameters()
            
            for param_name, controls in self.optimization_controls.items():
                vary = controls['vary'].value
                value = controls['value'].value
                min_val = controls['min'].value if controls['min'].value != 0 else None
                max_val = controls['max'].value if controls['max'].value != 0 else None
                
                model_params.add(param_name, value=value, vary=vary, min=min_val, max=max_val)
            
            # Get model specification  
            model_spec = self.variogram_model_map[self.model_selector.value]
            
            # Optimize
            best_fit_params = self.vgm.fit_variogram_model(
                model=model_spec,
                model_parameters=model_params,
                optimizer_kwargs=opt_kwargs
            )
            
            # Calculate fitted values for plotting
            from echopop.nwfsc_feat.variogram_models import variogram
            
            fitted_gamma = variogram(
                distance_lags=self.vgm.lags,
                model=model_spec,
                **best_fit_params
            )
            
            # Update plot with both theoretical and fitted lines
            theoretical_gamma = variogram(
                distance_lags=self.vgm.lags,
                model=model_spec,
                **{name: widget.value for name, widget in self.theoretical_param_widgets.items()}
            )
            
            self.update_plot_with_theoretical(theoretical_gamma, fitted_gamma)
            
            # Display results
            results_html = "<h3>Optimization Results</h3><table border='1' style='border-collapse: collapse;'>"
            results_html += "<tr><th>Parameter</th><th>Optimized Value</th><th>Standard Error</th></tr>"
            
            for param, value in best_fit_params.items():
                std_err = "N/A"  # Would need access to fit statistics for this
                results_html += f"<tr><td>{param}</td><td>{value:.6f}</td><td>{std_err}</td></tr>"
            
            results_html += "</table>"
            
            self.optimization_results.object = results_html
            
            self.optimization_status.object = f"""
            <p style="color: green;">
            <b>✓ Optimization completed successfully!</b><br>
            Model: {self.model_selector.value}<br>
            Converged: Yes
            </p>
            """
            
        except Exception as e:
            self.optimization_status.object = f"""
            <p style="color: red;">
            <b>✗ Optimization failed:</b><br>
            {str(e)}<br>
            {traceback.format_exc()}
            </p>
            """
    
    def create_optimization_controls(self):
        """Create optimization parameter controls"""
        controls = []
        self.optimization_controls = {}
        
        controls.append(pn.pane.Markdown("### Parameter Optimization Controls"))
        controls.append(pn.pane.Markdown("*Set vary=True to optimize, adjust min/max bounds as needed*"))
        
        for param_name, widget in self.theoretical_param_widgets.items():
            # Create control widgets for each parameter
            vary_checkbox = pn.widgets.Checkbox(name=f'Vary {param_name}', value=True, width=150)
            value_input = pn.widgets.NumberInput(name='Value', value=widget.value, format='0.6f', width=100)
            min_input = pn.widgets.NumberInput(name='Min', value=0.0, format='0.6f', width=100)
            max_input = pn.widgets.NumberInput(name='Max', value=0.0, format='0.6f', width=100)
            
            self.optimization_controls[param_name] = {
                'vary': vary_checkbox,
                'value': value_input, 
                'min': min_input,
                'max': max_input
            }
            
            param_row = pn.Row(
                pn.pane.HTML(f"<b>{param_name}:</b>", width=120),
                vary_checkbox, value_input, min_input, max_input
            )
            controls.append(param_row)
        
        self.optimization_params_pane.clear()
        self.optimization_params_pane.extend(controls)
    
    def show(self):
        """Display the GUI"""
        return self.tabs

# Create the GUI instance
variogram_gui = InteractiveVariogramGUI()

# Display instructions for usage
print("Interactive Variogram GUI created!")
print("\nUsage:")
print("1. gui = variogram_gui.show()  # Display the interface")
print("2. variogram_gui.set_data(your_dataframe, 'variable_name')  # Set your data")
print("3. Use the tabs to: Initialize → Compute Empirical → Fit Theoretical → Optimize")

# Display the GUI
variogram_gui.show()

In [64]:
import panel as pn
import holoviews as hv
import numpy as np
import pandas as pd
from lmfit import Parameters
import json
from echopop.nwfsc_feat.variogram_models import get_variogram_arguments

# Enable Panel extension for Jupyter
pn.extension('tabulator', 'bokeh')
hv.extension('bokeh')

# Define the variogram model parameter mapping
VARIOGRAM_MODEL_PARAMETER_MAP = {
    "Bessel-Exponential": ["bessel", "exponential"],
    "Bessel-Gaussian": ["bessel", "gaussian"], 
    "Cosine-Exponential": ["cosine", "exponential"],
    "Cosine-Gaussian": ["cosine", "gaussian"],
    "Cubic": "cubic",
    "Exponential": "exponential",
    "Exponential-Linear": ["exponential", "linear"],
    "Gaussian": "gaussian",
    "Gaussian-Linear": ["gaussian", "linear"],
    "J-Bessel": "jbessel",
    "K-Bessel": "kbessel", 
    "Linear": "linear",
    "Matérn": "matern",
    "Nugget": "nugget",
    "Pentaspherical": "pentaspherical",
    "Power law": "power",
    "Rational quadratic": "quadratic",
    "Cardinal sine": "sinc",
    "Spherical": "spherical",
}

# Global variables to store state
current_vgm = None
current_data = None
empirical_plot_pane = None
theoretical_params = {}
best_fit_params = None

# Create instructions tab
def create_instructions():
    instructions = pn.pane.Markdown("""
    # Variogram Analysis Interactive GUI
    
    ## Overview
    This interactive tool allows you to perform comprehensive variogram analysis with the following steps:
    
    ## Workflow
    
    ### Tab 2: Object Initialization
    - Set the **lag resolution** and **number of lags** for the variogram analysis
    - These parameters control the spatial resolution and extent of the analysis
    
    ### Tab 3: Compute Empirical Variogram
    - Select the variable to analyze
    - Configure azimuth filtering options
    - Generate an interactive plot showing:
      - X-axis: Lag distance
      - Y-axis: Semivariance (γ)
      - Point size: Varies with lag count
      - Hover tooltips: Lag #, lag counts, and semivariance values
    
    ### Tab 4: Fit Theoretical Model
    - Choose from available variogram models
    - Parameters dynamically update based on selected model
    - Black line shows theoretical fit on empirical plot
    
    ### Tab 5: Optimization
    - Specify optimization parameters as dictionary
    - Set parameter bounds and variation flags
    - Red line shows optimized fit
    - Display best-fit parameters with appropriate rounding
    
    ## Usage Notes
    - Make sure to load your data before proceeding
    - Each tab builds upon the previous steps
    - Results can be saved for further analysis
    """)
    return instructions

print("Simple Variogram GUI functions loaded!")

Simple Variogram GUI functions loaded!


In [65]:
# Tab 2: Object Initialization
def create_initialization_tab():
    global current_vgm
    
    # Create widgets
    lag_res_input = pn.widgets.FloatInput(name='Lag Resolution', value=0.002, start=0.001, end=0.01, step=0.001)
    n_lags_input = pn.widgets.IntInput(name='Number of Lags', value=30, start=5, end=100)
    status_pane = pn.pane.HTML("<p><i>Click 'Initialize Variogram' to create variogram object</i></p>")
    
    def initialize_variogram(event):
        global current_vgm
        try:
            current_vgm = variogram.Variogram(
                lag_resolution=lag_res_input.value,
                n_lags=n_lags_input.value,
                coordinate_names=("x", "y")
            )
            status_pane.object = f"<p><b>Success:</b> Variogram initialized with lag_resolution={lag_res_input.value}, n_lags={n_lags_input.value}</p>"
        except Exception as e:
            status_pane.object = f"<p><b>Error:</b> {str(e)}</p>"
    
    init_button = pn.widgets.Button(name="Initialize Variogram", button_type="primary")
    init_button.on_click(initialize_variogram)
    
    return pn.Column(
        pn.pane.Markdown("## Variogram Object Initialization"),
        lag_res_input,
        n_lags_input,
        init_button,
        status_pane,
        margin=10
    )

In [66]:
# Tab 3: Empirical Variogram
def create_empirical_variogram_plot(vgm):
    """Create interactive empirical variogram plot"""
    if not hasattr(vgm, 'empirical_variogram'):
        return pn.pane.Markdown("No empirical variogram data available")
    
    # Get empirical variogram data
    emp_data = vgm.empirical_variogram
    
    # Create DataFrame for plotting
    plot_data = pd.DataFrame({
        'lag_distance': emp_data['lag_distance'],
        'semivariance': emp_data['semivariance'], 
        'lag_count': emp_data['lag_count'],
        'lag_number': range(len(emp_data['lag_distance']))
    })
    
    # Create Holoviews scatter plot
    scatter = hv.Scatter(plot_data, kdims=['lag_distance'], 
                       vdims=['semivariance', 'lag_count', 'lag_number'])
    
    # Style the plot
    scatter = scatter.opts(
        size=hv.dim('lag_count').norm() * 20 + 5,  # Point size varies with lag count
        color='blue',
        alpha=0.7,
        tools=['hover'],
        width=600,
        height=400,
        xlabel='Lag Distance',
        ylabel='Semivariance (γ)',
        title='Empirical Variogram'
    )
    
    # Configure hover tooltips
    scatter = scatter.opts(
        hv.opts.Scatter(
            tools=[
                hv.bokeh.HoverTool(
                    tooltips=[
                        ('Lag #', '@lag_number'),
                        ('Lag Distance', '@lag_distance{0.000}'),
                        ('Semivariance', '@semivariance{0.000}'),
                        ('Lag Count', '@lag_count')
                    ]
                )
            ]
        )
    )
    
    return pn.pane.HoloViews(scatter, sizing_mode='stretch_width')

def create_empirical_tab():
    global current_vgm, current_data, empirical_plot_pane
    
    # Create widgets
    variable_input = pn.widgets.TextInput(name='Variable Name', value='biomass_density')
    azimuth_filter = pn.widgets.Checkbox(name='Azimuth Filter', value=True)
    azimuth_threshold = pn.widgets.FloatInput(name='Azimuth Angle Threshold', value=180.0, start=0, end=360)
    status_pane = pn.pane.HTML("<p><i>Set data first, then click 'Compute Empirical Variogram'</i></p>")
    
    # Plot pane placeholder
    empirical_plot_pane = pn.pane.HTML("<p>No plot available yet</p>")
    
    def compute_empirical(event):
        global current_vgm, current_data, empirical_plot_pane
        
        if current_vgm is None:
            status_pane.object = "<p><b>Error:</b> Initialize variogram object first</p>"
            return
        if current_data is None:
            status_pane.object = "<p><b>Error:</b> Set data first (use gui.set_data(df))</p>"
            return
        
        try:
            # Calculate empirical variogram
            current_vgm.calculate_empirical_variogram(
                data=current_data,
                variable=variable_input.value,
                azimuth_filter=azimuth_filter.value,
                azimuth_angle_threshold=azimuth_threshold.value
            )
            
            # Create interactive plot
            plot = create_empirical_variogram_plot(current_vgm)
            empirical_plot_pane.object = plot
            status_pane.object = "<p><b>Success:</b> Empirical variogram computed successfully</p>"
            
        except Exception as e:
            status_pane.object = f"<p><b>Error:</b> {str(e)}</p>"
    
    compute_button = pn.widgets.Button(name="Compute Empirical Variogram", button_type="primary")
    compute_button.on_click(compute_empirical)
    
    return pn.Column(
        pn.pane.Markdown("## Empirical Variogram Computation"),
        pn.pane.Markdown("*Note: Use `set_data(df)` function to set your DataFrame*"),
        variable_input,
        azimuth_filter,
        azimuth_threshold,
        compute_button,
        status_pane,
        empirical_plot_pane,
        margin=10
    )

In [67]:
# Tab 4: Theoretical Model
def get_model_parameters(model_name):
    """Get required parameters for the selected model"""
    try:
        model_key = VARIOGRAM_MODEL_PARAMETER_MAP[model_name]
        variogram_args, _ = get_variogram_arguments(model_key)
        param_names = [name for name in variogram_args.keys() if name != 'distance_lags']
        
        # Default parameter values
        defaults = {
            'nugget': 0.1,
            'sill': 1.0, 
            'correlation_range': 0.3,
            'hole_effect_range': 0.2,
            'decay_power': 1.5,
            'smoothness_parameter': 1.5,
            'shape_parameter': 2.0,
            'power_exponent': 1.0,
            'enhance_semivariance': False
        }
        
        return param_names, defaults
    except Exception as e:
        return [], {}

def create_theoretical_tab():
    global current_vgm, theoretical_params, empirical_plot_pane
    
    # Create widgets
    model_selector = pn.widgets.Select(
        name='Model Selection', 
        value='Exponential',
        options=list(VARIOGRAM_MODEL_PARAMETER_MAP.keys())
    )
    
    param_widgets_pane = pn.Column()
    status_pane = pn.pane.HTML("<p><i>Select model and set parameters, then click 'Fit Theoretical Model'</i></p>")
    
    def update_parameters(event):
        """Update parameter widgets when model changes"""
        model_name = model_selector.value
        param_names, defaults = get_model_parameters(model_name)
        
        # Clear previous widgets
        param_widgets_pane.clear()
        theoretical_params.clear()
        
        # Create new parameter widgets
        for param_name in param_names:
            default_val = defaults.get(param_name, 1.0)
            theoretical_params[param_name] = default_val
            
            if param_name == 'enhance_semivariance':
                widget = pn.widgets.Checkbox(name=param_name, value=default_val)
            else:
                widget = pn.widgets.FloatInput(name=param_name, value=default_val, step=0.001)
            
            # Update theoretical_params when widget changes
            def make_updater(pname, w):
                def update_param(event):
                    theoretical_params[pname] = w.value
                return update_param
            
            widget.param.watch(make_updater(param_name, widget), 'value')
            param_widgets_pane.append(widget)
    
    # Initialize with default model
    update_parameters(None)
    model_selector.param.watch(update_parameters, 'value')
    
    def fit_theoretical(event):
        """Fit theoretical model"""
        global current_vgm, empirical_plot_pane
        
        if current_vgm is None or not hasattr(current_vgm, 'empirical_variogram'):
            status_pane.object = "<p><b>Error:</b> Compute empirical variogram first</p>"
            return
        
        try:
            # Get the actual model key
            model_name = model_selector.value
            model_key = VARIOGRAM_MODEL_PARAMETER_MAP[model_name]
            
            # Create lmfit Parameters object
            lmfit_params = Parameters()
            for name, value in theoretical_params.items():
                lmfit_params.add(name, value=value, vary=False)
            
            # Fit theoretical model
            current_vgm.fit_variogram_model(
                model=model_key,
                model_parameters=lmfit_params,
                optimizer_kwargs={}
            )
            
            # Update plot with theoretical fit (black line)
            update_plot_with_theoretical()
            status_pane.object = "<p><b>Success:</b> Theoretical model fitted successfully</p>"
            
        except Exception as e:
            status_pane.object = f"<p><b>Error:</b> {str(e)}</p>"
    
    fit_button = pn.widgets.Button(name="Fit Theoretical Model", button_type="primary")
    fit_button.on_click(fit_theoretical)
    
    return pn.Column(
        pn.pane.Markdown("## Theoretical Model Fitting"),
        model_selector,
        param_widgets_pane,
        fit_button,
        status_pane,
        margin=10
    )

def update_plot_with_theoretical():
    """Update empirical plot with theoretical fit (black line)"""
    global current_vgm, empirical_plot_pane
    
    if empirical_plot_pane is None or current_vgm is None:
        return
    
    try:
        # Get data for both empirical and theoretical
        emp_data = current_vgm.empirical_variogram
        theo_data = current_vgm.theoretical_variogram
        
        # Create empirical scatter
        emp_df = pd.DataFrame({
            'lag_distance': emp_data['lag_distance'],
            'semivariance': emp_data['semivariance'], 
            'lag_count': emp_data['lag_count'],
            'lag_number': range(len(emp_data['lag_distance']))
        })
        
        scatter = hv.Scatter(emp_df, kdims=['lag_distance'], 
                           vdims=['semivariance', 'lag_count', 'lag_number'])
        scatter = scatter.opts(
            size=hv.dim('lag_count').norm() * 20 + 5,
            color='blue', alpha=0.7, tools=['hover'],
            width=600, height=400,
            xlabel='Lag Distance', ylabel='Semivariance (γ)',
            title='Empirical + Theoretical Variogram'
        )
        
        # Create theoretical curve
        theo_df = pd.DataFrame({
            'lag_distance': emp_data['lag_distance'],
            'semivariance': theo_data['semivariance']
        })
        
        curve = hv.Curve(theo_df, kdims=['lag_distance'], vdims=['semivariance'])
        curve = curve.opts(color='black', line_width=2, alpha=0.8)
        
        # Combine plots
        combined = (scatter * curve).opts(
            hv.opts.Scatter(
                tools=[hv.bokeh.HoverTool(
                    tooltips=[
                        ('Lag #', '@lag_number'),
                        ('Lag Distance', '@lag_distance{0.000}'),
                        ('Semivariance', '@semivariance{0.000}'),
                        ('Lag Count', '@lag_count')
                    ]
                )]
            )
        )
        
        empirical_plot_pane.object = pn.pane.HoloViews(combined, sizing_mode='stretch_width')
        
    except Exception as e:
        print(f"Error updating plot: {str(e)}")

In [68]:
# Tab 5: Optimization
def create_optimization_tab():
    global current_vgm, theoretical_params, best_fit_params, empirical_plot_pane
    
    # Create widgets
    opt_dict_input = pn.widgets.TextAreaInput(
        name='Optimization Parameters (JSON)', 
        value='{}',
        height=100,
        placeholder='e.g., {"max_nfev": 1000, "ftol": 1e-8}'
    )
    
    param_controls_pane = pn.Column()
    results_pane = pn.pane.HTML("<p><i>Parse optimization dict and create parameter controls first</i></p>")
    optimization_params = {}
    
    def parse_optimization_dict(event):
        """Parse the optimization dictionary"""
        try:
            opt_dict = json.loads(opt_dict_input.value if opt_dict_input.value else "{}")
            optimization_params.clear()
            optimization_params.update(opt_dict)
            results_pane.object = f"<p><b>Parsed:</b> {opt_dict}</p>"
        except json.JSONDecodeError as e:
            results_pane.object = f"<p><b>JSON Error:</b> {str(e)}</p>"
    
    def create_parameter_controls(event):
        """Create parameter variation controls"""
        if not theoretical_params:
            results_pane.object = "<p><b>Error:</b> Fit theoretical model first</p>"
            return
        
        param_controls_pane.clear()
        param_controls_pane.append(pn.pane.Markdown("### Parameter Optimization Controls"))
        
        # Store parameter control widgets
        param_control_widgets = {}
        
        for param_name, param_value in theoretical_params.items():
            # Create controls for each parameter
            vary_checkbox = pn.widgets.Checkbox(name=f'Vary {param_name}', value=True)
            min_input = pn.widgets.FloatInput(name='Min', value=0.0, step=0.001)
            max_input = pn.widgets.FloatInput(name='Max', value=param_value*2, step=0.001)
            value_input = pn.widgets.FloatInput(name='Value', value=param_value, step=0.001)
            
            param_control_widgets[param_name] = {
                'vary': vary_checkbox,
                'min': min_input,
                'max': max_input,
                'value': value_input
            }
            
            param_row = pn.Row(
                pn.pane.Markdown(f"**{param_name}:**"),
                vary_checkbox,
                pn.pane.Markdown("Value:"), value_input,
                pn.pane.Markdown("Min:"), min_input,
                pn.pane.Markdown("Max:"), max_input,
                sizing_mode='stretch_width'
            )
            param_controls_pane.append(param_row)
        
        # Store widgets for optimization function
        create_parameter_controls.widgets = param_control_widgets
        results_pane.object = "<p><b>Success:</b> Parameter controls created</p>"
    
    def run_optimization(event):
        """Run parameter optimization"""
        global current_vgm, best_fit_params, empirical_plot_pane
        
        if current_vgm is None or not hasattr(current_vgm, 'empirical_variogram'):
            results_pane.object = "<p><b>Error:</b> No empirical variogram computed</p>"
            return
        
        if not hasattr(create_parameter_controls, 'widgets'):
            results_pane.object = "<p><b>Error:</b> Create parameter controls first</p>"
            return
        
        try:
            # Get model key
            model_selector = [w for w in create_theoretical_tab().objects if hasattr(w, 'name') and w.name == 'Model Selection']
            if not model_selector:
                results_pane.object = "<p><b>Error:</b> Could not find model selector</p>"
                return
            
            # Use current model from theoretical_params instead
            model_key = None
            for model_name, key in VARIOGRAM_MODEL_PARAMETER_MAP.items():
                try:
                    param_names, _ = get_model_parameters(model_name)
                    if set(param_names) == set(theoretical_params.keys()):
                        model_key = key
                        break
                except:
                    continue
            
            if model_key is None:
                model_key = "exponential"  # fallback
            
            # Create optimized lmfit Parameters object
            opt_params = Parameters()
            widgets = create_parameter_controls.widgets
            
            for param_name in theoretical_params:
                if param_name in widgets:
                    vary = widgets[param_name]['vary'].value
                    value = widgets[param_name]['value'].value
                    min_val = widgets[param_name]['min'].value if vary else None
                    max_val = widgets[param_name]['max'].value if vary else None
                    
                    opt_params.add(param_name, value=value, vary=vary, min=min_val, max=max_val)
                else:
                    opt_params.add(param_name, value=theoretical_params[param_name], vary=True)
            
            # Run optimization
            best_fit_params = current_vgm.fit_variogram_model(
                model=model_key,
                model_parameters=opt_params,
                optimizer_kwargs=optimization_params
            )
            
            # Format best-fit parameters for display
            param_text = "<h3>Best-Fit Parameters:</h3><ul>"
            for name, param in best_fit_params.params.items():
                stderr = param.stderr if param.stderr is not None else 0.0
                param_text += f"<li><b>{name}:</b> {param.value:.4f} ± {stderr:.4f}</li>"
            param_text += "</ul>"
            
            # Update plot with optimized fit (red line)
            update_plot_with_optimization()
            results_pane.object = param_text
            
        except Exception as e:
            results_pane.object = f"<p><b>Error:</b> {str(e)}</p>"
    
    # Create buttons
    parse_button = pn.widgets.Button(name="Parse Dictionary", button_type="primary")
    parse_button.on_click(parse_optimization_dict)
    
    create_controls_button = pn.widgets.Button(name="Create Parameter Controls", button_type="primary") 
    create_controls_button.on_click(create_parameter_controls)
    
    optimize_button = pn.widgets.Button(name="Run Optimization", button_type="success")
    optimize_button.on_click(run_optimization)
    
    return pn.Column(
        pn.pane.Markdown("## Parameter Optimization"),
        pn.pane.Markdown("Enter optimization parameters as JSON dictionary:"),
        opt_dict_input,
        parse_button,
        create_controls_button,
        param_controls_pane,
        optimize_button,
        results_pane,
        margin=10
    )

def update_plot_with_optimization():
    """Update plot with optimized fit (red line)"""
    global current_vgm, empirical_plot_pane
    
    if empirical_plot_pane is None or current_vgm is None:
        return
    
    try:
        # Get data
        emp_data = current_vgm.empirical_variogram
        theo_data = current_vgm.theoretical_variogram  # This should be updated after optimization
        
        # Create empirical scatter
        emp_df = pd.DataFrame({
            'lag_distance': emp_data['lag_distance'],
            'semivariance': emp_data['semivariance'], 
            'lag_count': emp_data['lag_count'],
            'lag_number': range(len(emp_data['lag_distance']))
        })
        
        scatter = hv.Scatter(emp_df, kdims=['lag_distance'], 
                           vdims=['semivariance', 'lag_count', 'lag_number'])
        scatter = scatter.opts(
            size=hv.dim('lag_count').norm() * 20 + 5,
            color='blue', alpha=0.7, tools=['hover'],
            width=600, height=400,
            xlabel='Lag Distance', ylabel='Semivariance (γ)',
            title='Empirical + Theoretical + Optimized Variogram'
        )
        
        # Create optimized curve (red)
        opt_df = pd.DataFrame({
            'lag_distance': emp_data['lag_distance'],
            'semivariance': theo_data['semivariance']  # Updated after optimization
        })
        
        opt_curve = hv.Curve(opt_df, kdims=['lag_distance'], vdims=['semivariance'])
        opt_curve = opt_curve.opts(color='red', line_width=2, alpha=0.8)
        
        # Combine plots
        combined = (scatter * opt_curve).opts(
            hv.opts.Scatter(
                tools=[hv.bokeh.HoverTool(
                    tooltips=[
                        ('Lag #', '@lag_number'),
                        ('Lag Distance', '@lag_distance{0.000}'),
                        ('Semivariance', '@semivariance{0.000}'),
                        ('Lag Count', '@lag_count')
                    ]
                )]
            )
        )
        
        empirical_plot_pane.object = pn.pane.HoloViews(combined, sizing_mode='stretch_width')
        
    except Exception as e:
        print(f"Error updating optimized plot: {str(e)}")

# Helper function to set data
def set_data(data):
    """Set the data for variogram analysis"""
    global current_data
    current_data = data
    print(f"Data set with shape: {data.shape}")

# Main GUI creation function
def create_variogram_gui():
    """Create the main GUI with tabs"""
    tabs = pn.Tabs(
        ("Instructions", create_instructions()),
        ("Object Initialization", create_initialization_tab()), 
        ("Empirical Variogram", create_empirical_tab()),
        ("Theoretical Model", create_theoretical_tab()),
        ("Optimization", create_optimization_tab()),
        dynamic=True
    )
    
    return tabs

# Create the GUI
print("Creating Variogram GUI...")
variogram_gui = create_variogram_gui()
print("GUI created! Display with: variogram_gui")

Creating Variogram GUI...
GUI created! Display with: variogram_gui


In [81]:
# SIMPLE USAGE EXAMPLE
# 1. Set your data (replace with your dataframe):
set_data(df_nasc_no_age1)

# 2. Display the GUI:
variogram_gui.show()

Data set with shape: (9287, 25)
Launching server at http://localhost:52900


ERROR:tornado.application:Uncaught exception GET / (::1)
HTTPServerRequest(protocol='http', host='localhost:52900', method='GET', uri='/', version='HTTP/1.1', remote_ip='::1')
Traceback (most recent call last):
  File "c:\Users\Brandyn\miniconda3\envs\echopop_3_12\Lib\site-packages\tornado\web.py", line 1790, in _execute
    result = await result
             ^^^^^^^^^^^^
  File "c:\Users\Brandyn\miniconda3\envs\echopop_3_12\Lib\site-packages\panel\io\server.py", line 466, in get
    session = await self.get_session()
              ^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Brandyn\miniconda3\envs\echopop_3_12\Lib\site-packages\panel\io\server.py", line 356, in get_session
    session = await super().get_session()
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Brandyn\miniconda3\envs\echopop_3_12\Lib\site-packages\bokeh\server\views\session_handler.py", line 145, in get_session
    session = await self.application_context.create_session_if_needed(session_id, self.request, to

In [70]:
# Example of how to use the Variogram GUI
# This would integrate with your existing variogram workflow

# Create variogram object (this would be your modified version with gui=True parameter)
# vgm = variogram.Variogram(
#     lag_resolution=0.002,
#     n_lags=30, 
#     coordinate_names=("x", "y"),
#     gui=True  # This would trigger the GUI creation
# )

# Alternative approach - create GUI separately and connect to existing variogram
vgm_basic = variogram.Variogram(
    lag_resolution=0.002,
    n_lags=30,
    coordinate_names=("x", "y")
)

# Create GUI and connect to variogram object
gui = VariogramGUI(variogram_obj=vgm_basic)

# Set your data (replace with your actual dataframe)
# gui.set_data(df_nasc_no_age1)

# Display the interactive GUI
variogram_gui_app = gui.create_gui()
variogram_gui_app

AssertionError: 

Tabs(dynamic=True)
    [0] Markdown(str)
    [1] Column
        [0] Markdown(str)
        [1] Param(VariogramGUI, parameters=['lag_resolution', ...], widgets={'lag_resolution': <class ...})
        [2] Button(button_type='primary', name='Update Variogram')
        [3] ParamFunction(function, _pane=Markdown, defer_load=False)
    [2] Column
        [0] Markdown(str)
        [1] Param(VariogramGUI, parameters=['variable_name', ...])
        [2] Button(button_type='primary', name='Compute Empirical V...)
        [3] ParamFunction(function, _pane=Markdown, defer_load=False)
    [3] Column
        [0] Markdown(str)
        [1] Select(name='Model Selection', options=['Bessel-Exponential', ...], value='Exponential')
        [2] ParamFunction(function, _pane=Column, defer_load=False)
        [3] Button(button_type='primary', name='Fit Theoretical Model')
        [4] ParamFunction(function, _pane=Markdown, defer_load=False)
    [4] Column
        [0] Markdown(str)
        [1] Markdown(str)
    